# Introduction 

This code is inspired from [John's Article](https://www.codearmo.com/python-tutorial/options-trading-black-scholes-model)



# Importing the Libraries

In [ ]:
import numpy as np
from scipy.stats import norm
from matplotlib import pyplot

phi = norm.cdf

In [ ]:

def callOptionPrice(
        stockPrice: float,
        strikePrice: float, 
        riskFreeRate: float,
        volatility: float,
        timeToExpirationYr: float,
) -> float:
    d1 =  (np.log(stockPrice/strikePrice) + (riskFreeRate + volatility**2/2)*timeToExpirationYr)/(volatility*np.sqrt(timeToExpirationYr))
    d2 = d1 - volatility*np.sqrt(timeToExpirationYr)
    price = stockPrice*phi(d1) - strikePrice*np.exp(-riskFreeRate*timeToExpirationYr)*phi(d2)
    return price

def putOptionPrice(
        stockPrice: float,
        strikePrice: float, 
        riskFreeRate: float,
        volatility: float,
        timeToExpirationYr: float,
) -> float:
    d1 =  (np.log(stockPrice/strikePrice) + (riskFreeRate + volatility**2/2)*timeToExpirationYr)/(volatility*np.sqrt(timeToExpirationYr))
    d2 = d1 - volatility*np.sqrt(timeToExpirationYr)
    price = strikePrice*np.exp(-riskFreeRate*timeToExpirationYr)*phi(-d2) - stockPrice*phi(-d1)
    return price

In [ ]:
strikePrice = 100
riskFreeRate = 0.1
timeToExpirationYr = 1
volatility = 0.3
stockPrices = np.arange(60,140,0.1)

calls = [callOptionPrice(stockPrice, strikePrice, riskFreeRate, volatility, timeToExpirationYr) for stockPrice in stockPrices]
puts = [putOptionPrice(stockPrice, strikePrice, riskFreeRate, volatility, timeToExpirationYr) for stockPrice in stockPrices]
pyplot.plot(stockPrices,calls, label='Call Value')
pyplot.plot(stockPrices,puts, label='Put Value')
pyplot.xlabel('$S_0$')
pyplot.ylabel(' Value') 
pyplot.legend()

In [ ]:
strikePrice = 100
riskFreeRate = 0.1
timeToExpirationYr = 1
volatilities = np.arange(0.1, 1.5, 0.01)
stockPrice = 100

calls = [callOptionPrice(stockPrice, strikePrice, riskFreeRate, volatility, timeToExpirationYr) for volatility in volatilities]
puts = [putOptionPrice(stockPrice, strikePrice, riskFreeRate, volatility, timeToExpirationYr) for volatility in volatilities]
pyplot.plot(volatilities, calls, label='Call Value')
pyplot.plot(volatilities, puts, label='Put Value')
pyplot.xlabel('$\sigma$')
pyplot.ylabel(' Value') 
pyplot.legend()

In [ ]:
strikePrice = 100
riskFreeRate = 0.05
timeToExpirationYrs = np.arange(0, 10, 0.01)
volatilities = 0.3
stockPrice = 100

calls = [callOptionPrice(stockPrice, strikePrice, riskFreeRate, volatility, timeToExpirationYr) for timeToExpirationYr in timeToExpirationYrs]
puts = [putOptionPrice(stockPrice, strikePrice, riskFreeRate, volatility, timeToExpirationYr) for timeToExpirationYr in timeToExpirationYrs]
pyplot.plot(timeToExpirationYrs, calls, label='Call Value')
pyplot.plot(timeToExpirationYrs, puts, label='Put Value')
pyplot.xlabel('$T$ in years')
pyplot.ylabel(' Value') 
pyplot.legend()

In [1]:
from optionsPricingEngine.optionsCalculator import OptionCalculator

# 1. Initialize the OptionCalculator
calculator = OptionCalculator()

# 2. Select the desired option pricing model with its parameters
# For this example, we'll use the Black-Scholes model for a call option
calculator.select_model(
    modelName="BlackScholes",
    stockPrice=100,               # Current stock price
    strikePrice=105,              # Option strike price
    timeToExpirationYr=1,         # Time to expiration in years
    riskFreeRate=0.05,            # Annual risk-free rate
    volatility=0.2,               # Annual volatility
    dividendYield=0.03,           # Continuous dividend yield
    optionType="call"             # Option type (either "call" or "put")
)

# 3. Calculate the option price
price = calculator.calculate_price()
print(f"Option Price: {price:.2f}")

# 4. Calculate the option Greeks
greeks = calculator.calculate_greeks()
print(f"Greeks: {greeks}")

# 5. Calculate both the option price and its Greeks in one go
combined_result = calculator.calculate_price_and_greeks()
print(f"Combined Result: {combined_result}")

# If you need to switch to another model or option type, just use the select_model method again with new parameters.
# For instance, to use the GarmanKohlhagen model for a put option on a foreign currency:
calculator.select_model(
    modelName="GarmanKohlhagen",
    stockPrice=1.20,              # Current currency exchange rate (e.g., USD to EUR)
    strikePrice=1.25,             # Option strike price
    timeToExpirationYr=0.5,       # Time to expiration in years
    riskFreeRate=0.03,            # Annual domestic risk-free rate
    volatility=0.15,              # Annual volatility of the currency exchange rate
    foreignRiskFreeRate=0.02,     # Annual foreign risk-free rate
    optionType="put"              # Option type (either "call" or "put")
)

price = calculator.calculate_price_and_greeks()
print(f"Option Price for Foreign Currency Option: {price}")


Option Price: 6.51
Greeks: {'delta': 0.46821260591218816, 'gamma': 0.01933890045307749, 'vega': 38.677800906154985, 'theta': -4.478874363591334, 'rho': 40.314641814248006}
Combined Result: {'price': 6.506618776970811, 'delta': 0.46821260591218816, 'gamma': 0.01933890045307749, 'vega': 38.677800906154985, 'theta': -4.478874363591334, 'rho': 40.314641814248006}
Option Price for Foreign Currency Option: {'price': 0.07571712967603283, 'delta': -0.60597287398511, 'gamma': 2.9799497611421017, 'vega': 0.321834574203347, 'theta': -0.038731997752399744, 'rho': -0.4014422892290824}


In [2]:
from optionsPricingEngine.customRules.rules import rules
from optionsPricingEngine.recommendation.recommendationEngine import RecommendationEngine

engine = RecommendationEngine(rules=rules)

data = calculator.modelResults 
data.update(calculator.modelParams)
print(engine.recommend(data))


Put option has an intrinsic value of $0.050000000000000044. Exercising now would lead to a profit of $0.050000000000000044 per option. Consider buying if you anticipate the stock price will continue to fall. (Threshold: 1.25, Encountered: 1.2) 
[GarmanKohlhagen] The option has a longer time to expiration with 182.5 days left. Time decay is slower. This offers more time to make decisions based on other factors. (Threshold: 0.25, Encountered: 0.5) 
[GarmanKohlhagen] High gamma of 2.9799497611421017 indicates the currency option's delta is highly sensitive to changes in the currency pair. Such options can be affected rapidly by market movements, leading to potential quick gains or losses. (Threshold: 0.05, Encountered: 2.9799497611421017) 
[GarmanKohlhagen] The put option has high Vega sensitivity with a Vega of 0.321834574203347. This implies that for a 1% increase in implied volatility, the option's price will increase by approximately the Vega value. Consider this sensitivity when expe

In [3]:
calculator.modelParams, len(rules)

({'stockPrice': 1.2,
  'strikePrice': 1.25,
  'timeToExpirationYr': 0.5,
  'riskFreeRate': 0.03,
  'volatility': 0.15,
  'dividendYield': 0.0,
  'foreignRiskFreeRate': 0.02,
  'optionType': 'put',
  'modelName': 'GarmanKohlhagen',
  'delta': -0.60597287398511,
  'gamma': 2.9799497611421017,
  'vega': 0.321834574203347,
  'theta': -0.038731997752399744,
  'rho': -0.4014422892290824},
 47)